In [1]:
import os

import pandas as pd
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option("precision", 10)

In [3]:
sensor_data_path = "./sensor_data"

encoder_data = pd.read_csv(os.path.join(sensor_data_path,"encoder.csv"), names=["timestamp", "left_count", "right_count"])
fog_data = pd.read_csv(os.path.join(sensor_data_path,"fog.csv"), names=["timestamp", "delta_roll", "delta_pitch", "delta_yaw"])


# header names for lidar_data
lidar_angle = np.around(np.linspace(-5, 185, 286,),1).astype('str').tolist()
lidar_angle.insert(0, "timestamp")

lidar_data = pd.read_csv(os.path.join(sensor_data_path,"lidar.csv"),names=lidar_angle)

In [4]:
# first column represents the timestamp of the observation
'''
FOV: 190 (degree)
Start angle: -5 (degree)
End angle: 185 (degree)
Angular resolution: 0.666 (degree) # shoot a laser ray every 0.666 degree
Max range: 80 (meter)

* LiDAR rays with value 0.0 represent infinite range observations.
'''
print(lidar_data.shape)
# lidar_data.head(5)
# elment in 1:end columns is the range (distance in meter)

(115865, 287)


In [5]:
# Only consider points between the minimum and maximum range of the sensor.
# Measurements between 2m-75m are recommended to be included as valid data.
'''
lidar_data = lidar_data.drop(["timestamp"], axis = 1)
lidar_data.head()
lidar_data.plot.hist(bins=10)
plt.show()
'''

'\nlidar_data = lidar_data.drop(["timestamp"], axis = 1)\nlidar_data.head()\nlidar_data.plot.hist(bins=10)\nplt.show()\n'

In [6]:
##################################################################################################################

## Encoder

In [7]:
'''
Encoder calibrated parameter
Encoder resolution: 4096
Encoder left wheel diameter: 0.623479
Encoder right wheel diameter: 0.622806
Encoder wheel base: 1.52439

* The encoder data is stored as [timestamp, left count, right count].
'''
print(encoder_data.shape)
# encoder_data.head(10)

(116048, 3)


In [8]:
df = encoder_data.copy()
print(df.shape)
df = np.diff(df, axis=0)
df=np.vstack(([np.nan, np.nan, np.nan],df))
df.shape

(116048, 3)


(116048, 3)

In [9]:
new = {'dt': df[:,0]*1e-9, 'dlz': df[:,1], 'drz': df[:,2]}
new_df = pd.DataFrame(data=new)
# new_df.head()

In [10]:
res = 4096
dL = 0.623479
dR = 0.622806

new_df["timestamp"] = encoder_data["timestamp"]
new_df["distance_left"] = np.pi * dL/res * new_df["dlz"]
new_df["distance_right"] = np.pi * dR/res * new_df["drz"]
new_df['velocity_left'] = new_df["distance_left"] /  new_df["dt"]
new_df['velocity_right'] = new_df["distance_right"] /  new_df["dt"]
new_df['linear_velocity(m/s)'] = (new_df['velocity_left'] + new_df['velocity_right'])/2.0 
# new_df['linear_velocity(mph)'] = new_df['linear_velocity(m/s)']*2.237
#new_df.fillna(0, inplace=True)

print(new_df.shape)
new_df.head()

(116048, 9)


,dt,dlz,drz,timestamp,distance_left,distance_right,velocity_left,velocity_right,linear_velocity(m/s)
0,NaN,NaN,NaN,1544582648751852447,NaN,NaN,NaN,NaN,NaN
1,0.009897613,96.0,96.0,1544582648761750060,0.0459074308,0.0458578771,4.6382325483,4.6332259154,4.6357292318
2,0.009998718,96.0,96.0,1544582648771748778,0.0459074308,0.0458578771,4.5913316854,4.5863756785,4.5888536820
3,0.010016259,96.0,95.0,1544582648781765037,0.0459074308,0.0453801908,4.5832911037,4.5306526950,4.5569718994
4,0.010003228,96.0,95.0,1544582648791768265,0.0459074308,0.0453801908,4.5892616631,4.5365546834,4.5629081732


In [11]:
# print(sum(new_df['distance_left']), sum(new_df['distance_right']))
# new_df.describe()

In [12]:
##################################################################################################################

## FOG 

In [13]:
'''
FOG (Fiber Optic Gyro) extrinsic calibration parameter from vehicle
RPY(roll/pitch/yaw = XYZ extrinsic, degree), R(rotation matrix), T(translation matrix, meter)
RPY: 0 0 0
R: 1 0 0 0 1 0 0 0 1 
T: -0.335 -0.035 0.78

* The sensor measurements are stored as [timestamp, delta roll, delta pitch, delta yaw] in radians.
'''

print(fog_data.shape)
# fog_data.head(20)

(1160508, 4)


In [14]:
fog_data['dt'] = np.hstack((np.nan,np.diff(fog_data['timestamp']*1e-9)))
# fog_data.head(10)

In [15]:
#####################################################################################################

# sychronize

In [16]:
lidar_df = lidar_data.copy()
fog_df = fog_data.copy()
encoder_df = new_df.copy()
encoder_df.drop(["dt","dlz", "drz", "distance_left", "distance_right","velocity_left","velocity_right"], axis=1, inplace=True)

In [17]:
print(lidar_df.shape)
print(fog_df.shape)
print(encoder_df.shape)

(115865, 287)
(1160508, 5)
(116048, 2)


In [18]:
# lidar_df.head(20)

In [19]:
fog_df.head()

,timestamp,delta_roll,delta_pitch,delta_yaw,dt
0,1544582648743131772,-0.0000050486,0.0000275229,-0.0000119692,NaN
1,1544582648744114218,-0.0000073519,0.0000246436,-0.0000110483,0.0009825230
2,1544582648745131427,0.0000019153,0.0000220933,-0.0000103648,0.0010175705
3,1544582648746103145,0.0000166523,0.0000216797,-0.0000098693,0.0009713173
4,1544582648747106429,0.0000310455,0.0000226423,-0.0000097198,0.0010032654


In [20]:
encoder_df.head()

,timestamp,linear_velocity(m/s)
0,1544582648751852447,NaN
1,1544582648761750060,4.6357292318
2,1544582648771748778,4.5888536820
3,1544582648781765037,4.5569718994
4,1544582648791768265,4.5629081732


In [21]:
fog_df['timestamp']=np.round(fog_df['timestamp']*1e-6).astype("int64")

In [22]:
fog_df['wt'] = fog_df['delta_yaw'] / fog_df['dt'] 

In [23]:
print(fog_df.shape)
fog_df.head(10)

(1160508, 6)


,timestamp,delta_roll,delta_pitch,delta_yaw,dt,wt
0,1544582648743,-0.0000050486,0.0000275229,-0.0000119692,NaN,NaN
1,1544582648744,-0.0000073519,0.0000246436,-0.0000110483,0.0009825230,-0.0112447759
2,1544582648745,0.0000019153,0.0000220933,-0.0000103648,0.0010175705,-0.0101858691
3,1544582648746,0.0000166523,0.0000216797,-0.0000098693,0.0009713173,-0.0101607524
4,1544582648747,0.0000310455,0.0000226423,-0.0000097198,0.0010032654,-0.0096881970
5,1544582648748,0.0000394436,0.0000223999,-0.0000096258,0.0010201931,-0.0094352547
6,1544582648749,0.0000353109,0.0000206480,-0.0000097710,0.0009782314,-0.0099884106
7,1544582648750,0.0000211805,0.0000173981,-0.0000095623,0.0010268688,-0.0093121180
8,1544582648751,0.0000026081,0.0000128376,-0.0000088801,0.0009679794,-0.0091738255
9,1544582648752,-0.0000149339,0.0000061556,-0.0000087522,0.0010323524,-0.0084779171


In [24]:
encoder_df['timestamp']=np.round(encoder_df['timestamp']*1e-6).astype("int64")

In [25]:
print(encoder_df.shape)
encoder_df.head(10)

(116048, 2)


,timestamp,linear_velocity(m/s)
0,1544582648752,NaN
1,1544582648762,4.6357292318
2,1544582648772,4.5888536820
3,1544582648782,4.5569718994
4,1544582648792,4.5629081732
5,1544582648802,4.5970392340
6,1544582648812,4.6041201653
7,1544582648822,4.6125740375
8,1544582648832,4.6152294822
9,1544582648842,4.6983616970


In [26]:
mergedf_fog_encoder_left =fog_df.merge(encoder_df, how="left",)

In [27]:
print(mergedf_fog_encoder_left.shape)

(1160508, 7)


In [28]:
mergedf_fog_encoder_left.head()

,timestamp,delta_roll,delta_pitch,delta_yaw,dt,wt,linear_velocity(m/s)
0,1544582648743,-0.0000050486,0.0000275229,-0.0000119692,NaN,NaN,NaN
1,1544582648744,-0.0000073519,0.0000246436,-0.0000110483,0.0009825230,-0.0112447759,NaN
2,1544582648745,0.0000019153,0.0000220933,-0.0000103648,0.0010175705,-0.0101858691,NaN
3,1544582648746,0.0000166523,0.0000216797,-0.0000098693,0.0009713173,-0.0101607524,NaN
4,1544582648747,0.0000310455,0.0000226423,-0.0000097198,0.0010032654,-0.0096881970,NaN


In [29]:
mergedf_fog_encoder_left.shape

(1160508, 7)

In [30]:
mergedf_fog_encoder_left["linear_velocity(m/s)"].unique()

array([       nan, 4.63572923, 4.58885368, ..., 6.80738106, 6.81220312,
       6.81032775])

In [31]:
mergedf_fog_encoder_left.isnull().any()

timestamp               False
delta_roll              False
delta_pitch             False
delta_yaw               False
dt                       True
wt                       True
linear_velocity(m/s)     True
dtype: bool

In [32]:
mergedf_fog_encoder_left.fillna(method='bfill',inplace=True)
# mergedf_fog_encoder_left.head(100)

In [33]:
mergedf_fog_encoder_left.tail(20)

,timestamp,delta_roll,delta_pitch,delta_yaw,dt,wt,linear_velocity(m/s)
1160488,1544583809212,0.0000012073,-0.0000025920,-0.0000007287,0.0009984970,-0.0007298052,6.8122031193
1160489,1544583809213,0.0000056566,0.0000038938,0.0000000542,0.0009829998,0.0000551316,6.8122031193
1160490,1544583809214,0.0000067532,0.0000093901,0.0000002716,0.0010194778,0.0002663815,6.8122031193
1160491,1544583809215,0.0000045995,0.0000124650,0.0000010562,0.0010013580,0.0010547380,6.8122031193
1160492,1544583809216,-0.0000025343,0.0000116023,0.0000012374,0.0010187626,0.0012146132,6.8122031193
1160493,1544583809217,-0.0000095242,0.0000078488,0.0000017970,0.0009527206,0.0018861535,6.8122031193
1160494,1544583809218,-0.0000128561,0.0000040793,0.0000018875,0.0010383129,0.0018178100,6.8122031193
1160495,1544583809219,-0.0000131552,0.0000022325,0.0000017695,0.0009620190,0.0018394117,6.8103277505
1160496,1544583809220,-0.0000076909,0.0000031329,0.0000018200,0.0010004044,0.0018192365,6.8103277505
1160497,1544583809221,-0.0000007763,0.0000060454,0.0000015833,0.0010015965,0.0015807852,6.8103277505


In [34]:
mergedf_fog_encoder_left.isnull().any()

timestamp               False
delta_roll              False
delta_pitch             False
delta_yaw               False
dt                      False
wt                      False
linear_velocity(m/s)     True
dtype: bool

In [35]:
mergedf_fog_encoder_left.fillna(method='ffill',inplace=True)
print(mergedf_fog_encoder_left.shape)
mergedf_fog_encoder_left.head(100)

(1160508, 7)


,timestamp,delta_roll,delta_pitch,delta_yaw,dt,wt,linear_velocity(m/s)
0,1544582648743,-0.0000050486,0.0000275229,-0.0000119692,0.0009825230,-0.0112447759,4.6357292318
1,1544582648744,-0.0000073519,0.0000246436,-0.0000110483,0.0009825230,-0.0112447759,4.6357292318
2,1544582648745,0.0000019153,0.0000220933,-0.0000103648,0.0010175705,-0.0101858691,4.6357292318
3,1544582648746,0.0000166523,0.0000216797,-0.0000098693,0.0009713173,-0.0101607524,4.6357292318
4,1544582648747,0.0000310455,0.0000226423,-0.0000097198,0.0010032654,-0.0096881970,4.6357292318
5,1544582648748,0.0000394436,0.0000223999,-0.0000096258,0.0010201931,-0.0094352547,4.6357292318
6,1544582648749,0.0000353109,0.0000206480,-0.0000097710,0.0009782314,-0.0099884106,4.6357292318
7,1544582648750,0.0000211805,0.0000173981,-0.0000095623,0.0010268688,-0.0093121180,4.6357292318
8,1544582648751,0.0000026081,0.0000128376,-0.0000088801,0.0009679794,-0.0091738255,4.6357292318
9,1544582648752,-0.0000149339,0.0000061556,-0.0000087522,0.0010323524,-0.0084779171,4.6357292318


In [36]:
mergedf_fog_encoder_left.isnull().any()

timestamp               False
delta_roll              False
delta_pitch             False
delta_yaw               False
dt                      False
wt                      False
linear_velocity(m/s)    False
dtype: bool

In [37]:
mergedf_fog_encoder_left.drop(["delta_roll", "delta_pitch",], axis=1, inplace=True)

In [38]:
print(mergedf_fog_encoder_left.shape)
mergedf_fog_encoder_left.head()

(1160508, 5)


,timestamp,delta_yaw,dt,wt,linear_velocity(m/s)
0,1544582648743,-0.0000119692,0.0009825230,-0.0112447759,4.6357292318
1,1544582648744,-0.0000110483,0.0009825230,-0.0112447759,4.6357292318
2,1544582648745,-0.0000103648,0.0010175705,-0.0101858691,4.6357292318
3,1544582648746,-0.0000098693,0.0009713173,-0.0101607524,4.6357292318
4,1544582648747,-0.0000097198,0.0010032654,-0.0096881970,4.6357292318


In [39]:
# All in one

In [40]:
lidar_df["timestamp"] = np.round(lidar_df['timestamp']*1e-6).astype("int64")

In [41]:
merge_all_left=mergedf_fog_encoder_left.merge(lidar_df,on="timestamp", how="left",)

In [42]:
merge_all_inner=mergedf_fog_encoder_left.merge(lidar_df,on="timestamp", how="inner",)

In [43]:
print("inner:",merge_all_inner.shape)
print("left:",merge_all_left.shape)


inner: (115833, 291)
left: (1160508, 291)


In [44]:
# merge_all_inner[50000:50500]

In [45]:
merge_all_inner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115833 entries, 0 to 115832
Columns: 291 entries, timestamp to 185.0
dtypes: float64(290), int64(1)
memory usage: 258.1 MB


In [46]:
merge_all_left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1160508 entries, 0 to 1160507
Columns: 291 entries, timestamp to 185.0
dtypes: float64(290), int64(1)
memory usage: 2.5 GB


In [47]:
# merge_all_left[10000:10500]

In [48]:
merge_all_left.shape

(1160508, 291)

In [49]:
# np.isnan(np.sum(lidar_data[0,:]))

In [50]:
merge_all_left.head()

,timestamp,delta_yaw,dt,wt,linear_velocity(m/s),-5.0,-4.3,-3.7,-3.0,-2.3,-1.7,-1.0,-0.3,0.3,1.0,1.7,2.3,3.0,3.7,4.3,5.0,5.7,6.3,7.0,7.7,8.3,9.0,9.7,10.3,11.0,11.7,12.3,13.0,13.7,14.3,15.0,15.7,16.3,17.0,17.7,18.3,19.0,19.7,20.3,21.0,21.7,22.3,23.0,23.7,24.3,25.0,25.7,26.3,27.0,27.7,28.3,29.0,29.7,30.3,31.0,31.7,32.3,33.0,33.7,34.3,35.0,35.7,36.3,37.0,37.7,38.3,39.0,39.7,40.3,41.0,41.7,42.3,43.0,43.7,44.3,45.0,45.7,46.3,47.0,47.7,48.3,49.0,49.7,50.3,51.0,51.7,52.3,53.0,53.7,54.3,55.0,55.7,56.3,57.0,57.7,58.3,59.0,59.7,60.3,61.0,61.7,62.3,63.0,63.7,64.3,65.0,65.7,66.3,67.0,67.7,68.3,69.0,69.7,70.3,71.0,71.7,72.3,73.0,73.7,74.3,75.0,75.7,76.3,77.0,77.7,78.3,79.0,79.7,80.3,81.0,81.7,82.3,83.0,83.7,84.3,85.0,85.7,86.3,87.0,87.7,88.3,89.0,89.7,90.3,91.0,91.7,92.3,93.0,93.7,94.3,95.0,95.7,96.3,97.0,97.7,98.3,99.0,99.7,100.3,101.0,101.7,102.3,103.0,103.7,104.3,105.0,105.7,106.3,107.0,107.7,108.3,109.0,109.7,110.3,111.0,111.7,112.3,113.0,113.7,114.3,115.0,115.7,116.3,117.0,117.7,118.3,119.0,119.7,120.3,121.0,121.7,122.3,123.0,123.7,124.3,125.0,125.7,126.3,127.0,127.7,128.3,129.0,129.7,130.3,131.0,131.7,132.3,133.0,133.7,134.3,135.0,135.7,136.3,137.0,137.7,138.3,139.0,139.7,140.3,141.0,141.7,142.3,143.0,143.7,144.3,145.0,145.7,146.3,147.0,147.7,148.3,149.0,149.7,150.3,151.0,151.7,152.3,153.0,153.7,154.3,155.0,155.7,156.3,157.0,157.7,158.3,159.0,159.7,160.3,161.0,161.7,162.3,163.0,163.7,164.3,165.0,165.7,166.3,167.0,167.7,168.3,169.0,169.7,170.3,171.0,171.7,172.3,173.0,173.7,174.3,175.0,175.7,176.3,177.0,177.7,178.3,179.0,179.7,180.3,181.0,181.7,182.3,183.0,183.7,184.3,185.0
0,1544582648743,-0.0000119692,0.0009825230,-0.0112447759,4.6357292318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1544582648744,-0.0000110483,0.0009825230,-0.0112447759,4.6357292318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [54]:
merge_all_inner.head()

,timestamp,delta_yaw,dt,wt,linear_velocity(m/s),-5.0,-4.3,-3.7,-3.0,-2.3,-1.7,-1.0,-0.3,0.3,1.0,1.7,2.3,3.0,3.7,4.3,5.0,5.7,6.3,7.0,7.7,8.3,9.0,9.7,10.3,11.0,11.7,12.3,13.0,13.7,14.3,15.0,15.7,16.3,17.0,17.7,18.3,19.0,19.7,20.3,21.0,21.7,22.3,23.0,23.7,24.3,25.0,25.7,26.3,27.0,27.7,28.3,29.0,29.7,30.3,31.0,31.7,32.3,33.0,33.7,34.3,35.0,35.7,36.3,37.0,37.7,38.3,39.0,39.7,40.3,41.0,41.7,42.3,43.0,43.7,44.3,45.0,45.7,46.3,47.0,47.7,48.3,49.0,49.7,50.3,51.0,51.7,52.3,53.0,53.7,54.3,55.0,55.7,56.3,57.0,57.7,58.3,59.0,59.7,60.3,61.0,61.7,62.3,63.0,63.7,64.3,65.0,65.7,66.3,67.0,67.7,68.3,69.0,69.7,70.3,71.0,71.7,72.3,73.0,73.7,74.3,75.0,75.7,76.3,77.0,77.7,78.3,79.0,79.7,80.3,81.0,81.7,82.3,83.0,83.7,84.3,85.0,85.7,86.3,87.0,87.7,88.3,89.0,89.7,90.3,91.0,91.7,92.3,93.0,93.7,94.3,95.0,95.7,96.3,97.0,97.7,98.3,99.0,99.7,100.3,101.0,101.7,102.3,103.0,103.7,104.3,105.0,105.7,106.3,107.0,107.7,108.3,109.0,109.7,110.3,111.0,111.7,112.3,113.0,113.7,114.3,115.0,115.7,116.3,117.0,117.7,118.3,119.0,119.7,120.3,121.0,121.7,122.3,123.0,123.7,124.3,125.0,125.7,126.3,127.0,127.7,128.3,129.0,129.7,130.3,131.0,131.7,132.3,133.0,133.7,134.3,135.0,135.7,136.3,137.0,137.7,138.3,139.0,139.7,140.3,141.0,141.7,142.3,143.0,143.7,144.3,145.0,145.7,146.3,147.0,147.7,148.3,149.0,149.7,150.3,151.0,151.7,152.3,153.0,153.7,154.3,155.0,155.7,156.3,157.0,157.7,158.3,159.0,159.7,160.3,161.0,161.7,162.3,163.0,163.7,164.3,165.0,165.7,166.3,167.0,167.7,168.3,169.0,169.7,170.3,171.0,171.7,172.3,173.0,173.7,174.3,175.0,175.7,176.3,177.0,177.7,178.3,179.0,179.7,180.3,181.0,181.7,182.3,183.0,183.7,184.3,185.0
0,1544582648751,-0.0000088801,0.0009679794,-0.0091738255,4.6357292318,30.3900012970,30.6480007172,32.2060012817,32.1820030212,32.9260025024,33.3610000610,37.9390029907,33.0780029297,37.7340011597,38.0100021362,38.2470016479,31.7120018005,39.5069999695,30.7470016479,30.6480007172,25.3240013123,0.0000000000,35.2250022888,9.6190004349,9.9150009155,25.6530017853,9.6400003433,9.6610002518,25.4010009766,24.1070003510,23.9710006714,28.4210014343,0.0000000000,0.0000000000,36.1060028076,41.4320030212,0.0000000000,0.0,0.0,0.0000000000,26.9100017548,0.0,0.0,0.0,0.0,0.0,0.0000000000,10.7310009003,10.4230003357,10.2610006332,10.5210008621,38.7590026855,0.0000000000,0.0,0.0,0.0,0.0,60.2400016785,0.0,0.0,0.0000000000,56.6020011902,56.5020027161,55.6280021667,0.0000000000,53.6440010071,53.9360008240,54.4200019836,0.0000000000,55.4470024109,55.9870033264,56.5380020142,57.1330032349,57.7100028992,58.3480033875,58.9750022888,59.6350021362,60.3420028687,61.0460014343,61.7800025940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000000000,0.0,0.0,0.0000000000,0.0000000000,9.0640001297,9.1020002365,9.1140003204,6.1890001297,6.1870002747,6.2030000687,6.1530003548,5.0030002594,4.9850001335,5.0890002251,4.4690003395,5.2720003128,5.2670001984,4.3300004005,4.3470001221,18.4430007935,4.3220000267,4.2790002823,4.3550000191,4.3949999809,4.4420003891,4.4700002670,4.5410003662,4.5720000267,4.6480002403,5.2860002518,5.3450002670,10.7810001373,4.6200003624,4.6590003967,4.6060004234,3.8420002460,3.9070003033,4.6320004463,4.1750001907,4.1560001373,4.9770002365,4.1710000038,5.0830001831,3.9700002670,3.9750001431,3.9170002937,3.5300002098,4.9010000229,4.8780002594,3.9670002460,3.9900002480,5.2030000687,6.1760001183,6.3070001602,5.1550002098,5.1180000305,3.7390000820,8.3840007782,3.8240001202,3.9510002136,3.8030002117,3.7330002785,3.7510001659,3.7990002632,10.4940004349,10.5940008163,10.5090007782,10.9150009155,10.0940008163,10.1500005722,11.4820003510,10.3490009308,10.0890007019,9.1830005646,9.0270004272,11.7560005188,13.4690008163,0.0000000000,7.9400005341,7.95400047

# Output to .csv

In [51]:
#mergedf_fog_encoder_left.to_csv("sync_fog_encoder_left.csv",index=False)

In [55]:
# merge_all_inner.to_csv("sync_merge_all_inner.csv",index=False)

In [53]:
# merge_all_left.to_csv("sync_merge_all_left.csv",index=False)